In [18]:
import warnings
warnings.filterwarnings("ignore")

from data_loading import *
from xai_agg import *

import pandas as pd
import numpy as np

from scipy.stats import pearsonr, spearmanr, kendalltau

In [19]:
dataset_name, preprocessed_data, categorical_features, X, y, X_train, X_test, y_train, y_test, clf = load_pakdd2010_data()

Accuracy: 0.7466666666666667
ROC AUC: 0.5158014399393709


In [20]:
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
evaluator.init()

Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2130 - val_loss: 1.1088
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: 1.2059 - val_loss: 1.0603
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - loss: 1.1921 - val_loss: 1.0120
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - loss: 1.3268 - val_loss: 0.9662
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 1.0548 - val_loss: 0.9278
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 0.9555 - val_loss: 0.8973
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 1.0004 - val_loss: 0.8737
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 0.9619 - val_loss: 0.8548
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - loss: 0.9319 - val_loss: 0.8393
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - loss: 0.9708 - val_loss: 0.8260
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - loss: 0.9339 - val_loss: 0.8143
Epoch 12/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 

### Misc

In [21]:
evaluator.faithfullness_correlation(ShapTabularTreeWrapper, X_test.iloc[0])

0.6915117682202048

# Generating metric vectors and calculating correlation between them

In [22]:
# Getting a few reanknings
exps = [LimeWrapper(clf, X_train, categorical_features), ShapTabularTreeWrapper(clf, X_train, categorical_features), AnchorWrapper(clf, X_train, categorical_features)]

indexes = np.random.choice(X_test.index, 100, replace=False)


In [23]:
import concurrent.futures

comp_arr = [] # original, new1, new2, ...

def process_index(idx):
    local_comp_arr = []
    for exp in exps:
        row = []
        # explanation = exp.explain_instance(X_test.loc[idx])
        
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, iterations=100, len_subset=1))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, iterations=100, len_subset=1))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="sum", iterations=100, len_subset=1))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="percentile", iterations=100, len_subset=1))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="avg", iterations=100, len_subset=1))
        # row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="inverse", iterations=100, len_subset=1))
        
        # row.append(evaluator.complexity(exp, X_test.loc[idx], explanation=explanation))
        # row.append(evaluator.nrc(exp, X_test.loc[idx], explanation=explanation))
        
        row.append(evaluator._sensitivity_sequential(exp, X_test.loc[idx], method='mean_absolute', iterations=1))
        row.append(evaluator._sensitivity_sequential(exp, X_test.loc[idx], method="spearman", iterations=1))
        
        local_comp_arr.append(row)
    return local_comp_arr
    
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(process_index, indexes))

for result in results:
    comp_arr.extend(result)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

In [24]:
comp_arr

[[0.0008856545852878739, 0.7800586510263928],
 [0.0008380033888166596, 0.9933896437752479],
 [0.03145833333333333, 0.7184212081070995],
 [0.001175577095221063, 0.8665689149560116],
 [0.0007403652000692767, 0.9933896437752479],
 [0.11561197916666666, 0.6330032127588345],
 [0.0009679633416458217, 0.8676686217008798],
 [0.0008412419710094097, 0.9827396254131471],
 [0.047356770833333325, 0.8296959777368934],
 [0.000943235226089521, 0.8123167155425219],
 [0.0007939854862945769, 0.9878810135879543],
 [0.10016927083333332, 0.2328354796127735],
 [0.0009556669365518614, 0.8082844574780058],
 [0.0009037461124487224, 0.9944913698127067],
 [0.05838541666666667, 0.1721244416916042],
 [0.0010000650151084868, 0.8097507331378297],
 [0.0007141438283512258, 0.9786999632757988],
 [0.07940104166666667, 0.4228033098995545],
 [0.0009607479523114388, 0.8383431085043986],
 [0.0008343636937755149, 0.9893499816378994],
 [0.0, 0.7184212081070995],
 [0.0008898152004622613, 0.8328445747800586],
 [0.000689851891833

In [25]:

# from scipy.stats import spearmanr

# comp_arr = np.array(comp_arr)
# rank_algs = ["faithfulness (itself)", "rank_based_fairthfulness:sum", "rank_based_fairthfulness:percentile", "rank_based_fairthfulness:avg", "rank_based_fairthfulness:inverse"]
# # rank_algs = ["NRC"]

# pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
# spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]
# kendall_corr = [kendalltau(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

# correlation_df = pd.DataFrame({
#     'Rank Algorithm': rank_algs,
#     'Pearson Correlation': pearson_corr,
#     'Spearman Correlation': spearman_corr,
#     'Kendall Correlation': kendall_corr
# })

# print(dataset_name)
# print("The following are the correlations between the faithfullness metric and... (abs fixed, subset_len=1, iterations=100)")
# display(correlation_df)

In [26]:
from scipy.stats import spearmanr

comp_arr = np.array(comp_arr)
rank_algs = ["stability:spearman"]

pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]
kendall_corr = [kendalltau(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

correlation_df = pd.DataFrame({
    'Rank Algorithm': rank_algs,
    'Pearson Correlation': pearson_corr,
    'Spearman Correlation': spearman_corr,
    'Kendall Correlation': kendall_corr
})

print(dataset_name)
print("The following are the correlations between the sensitivity (mean-absolute) metric and...")
display(correlation_df)

PAKDD2010
The following are the correlations between the sensitivity (mean-absolute) metric and...


,Rank Algorithm,Pearson Correlation,Spearman Correlation,Kendall Correlation
0,stability:spearman,-0.712735,-0.669238,-0.475482
